# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [26]:
import os, re
import glob
import json
import psycopg2
import pandas as pd
from sql_queries import *

In [27]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=*******")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [32]:
cur.execute("SELECT * FROM songplays;")
pd.DataFrame(cur.fetchall())

,0,1,2,3,4,5,6,7,8
0,1,00:25:43.537327,91,free,None,None,829,"Dallas-Fort Worth-Arlington, TX",Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...
1,2,00:25:43.540121,73,paid,None,None,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,3,00:25:43.540368,73,paid,None,None,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,4,00:25:43.540625,73,paid,None,None,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4,5,00:25:43.540856,73,paid,None,None,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
5,6,00:25:43.541072,73,paid,None,None,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
6,7,00:25:43.541299,73,paid,None,None,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
7,8,00:25:43.541520,73,paid,None,None,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
8,9,00:25:43.541897,73,paid,None,None,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
9,10,00:25:43.541993,73,paid,None,None,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [41]:
song_files = get_files("data/song_data")
# song_files

In [42]:
filepath = '/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [43]:
# with open(filepath, "r") as read_file:
#     data = json.load(read_file)
# df = pd.DataFrame([data])
# df.head(n=10)
# df.dtypes
df = pd.read_json(filepath, lines=True)
df.head()
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [44]:
df['title'] = df['title'].map(lambda x: re.sub(r'\W+', ' ', x))
song_data = df[["song_id", "title", "artist_id", "year", "duration"]]
song_data = list(song_data.values[0])
song_data

['SOMZWCG12A8C13C480', 'I Didn t Mean To', 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
df['artist_name'] = df['artist_name'].map(lambda x: re.sub(r'\W+', ' ', x))
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
artist_data = list(artist_data.values[0])
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data")

In [13]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [35]:
df = pd.read_json(filepath, lines=True)
df['artist'] = df['artist'].map(lambda x: re.sub(r'\W+', ' ', str(x)))
df['song'] = df['song'].map(lambda x: re.sub(r'\W+', ' ', str(x)))

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df = df[df['page'] == "NextSong"]
# df.head(5)

In [16]:
t = pd.DatetimeIndex(df['ts'])
t.weekday

Int64Index([3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
            ...
            3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
           dtype='int64', name='ts', length=330)

In [49]:
time_data = [t.time, t.hour, t.day, t.weekofyear, t.month, t.year, t.weekday]
column_labels = ("timestamp","hour", "day", "week_of_year", "month", "year", "weekday")

In [52]:
timeDict = {}
for i in range(len(time_data)):
    timeDict[column_labels[i]] = time_data[i]
time_df = pd.DataFrame.from_dict(timeDict)
time_df

,timestamp,hour,day,week_of_year,month,year,weekday
0,00:25:43.537327,0,1,1,1,1970,3
1,00:25:43.540121,0,1,1,1,1970,3
2,00:25:43.540368,0,1,1,1,1970,3
3,00:25:43.540625,0,1,1,1,1970,3
4,00:25:43.540856,0,1,1,1,1970,3
5,00:25:43.541072,0,1,1,1,1970,3
6,00:25:43.541299,0,1,1,1,1970,3
7,00:25:43.541520,0,1,1,1,1970,3
8,00:25:43.541897,0,1,1,1,1970,3
9,00:25:43.541993,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
#     print("\ni & row: ", i, " row: ", row, " time_table_insert: ", time_table_insert)
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = df[['userId','firstName', 'lastName', 'gender', 'level']]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson s Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight Feat Slick Rick,200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won t Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    print(pd.to_datetime(row.ts).time())
    # insert songplay record
    songplay_data = (str(pd.to_datetime(row.ts).time()), str(row.userId),str(row.level), str(songid), str(artistid), str(row.sessionId), str(row.location), str(row.userAgent))
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

row:  Jim Henson s Dead  artist:  Stephen Lynch  row.length:  182.85669
results:  None
00:25:43.537327
row:  Shell Shock  artist:  Manowar  row.length:  247.562
results:  None
00:25:43.540121
row:  Women Lose Weight Feat Slick Rick   artist:  Morcheeba  row.length:  257.41016
results:  None
00:25:43.540368
row:  Won t Go Home Without You  artist:  Maroon 5  row.length:  231.23546
results:  None
00:25:43.540625
row:  Hey_ Soul Sister  artist:  Train  row.length:  216.76363
results:  None
00:25:43.540856
row:  I m In Miami Bitch  artist:  LMFAO  row.length:  227.99628
results:  None
00:25:43.541072
row:  Sexy Bitch  artist:  DJ Dizzy  row.length:  221.1522
results:  None
00:25:43.541299
row:  The Cure The Cause Dennis Ferrer Remix   artist:  Fish Go Deep Tracey K  row.length:  377.41669
results:  None
00:25:43.541520
row:  Staring At Me  artist:  M83  row.length:  96.1824
results:  None
00:25:43.541897
row:  The Second You Sleep  artist:  Saybia  row.length:  257.25342
results:  None
00:

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [24]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.